In [2]:
from openai.embeddings_utils import get_embedding
import os
import openai
import json
from Keys import openai_keys
import re
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn import tree
from sklearn.cluster import KMeans
from ast import literal_eval
import numpy as np
import random
openai.organization = openai_keys['organization']
openai.api_key = openai_keys['api_key']
embedding_model = "text-embedding-ada-002"

In [42]:
with open('Datasets/CDs_and_Vinyl_500k.json', 'r') as json_file:
    data = json.load(json_file)
random.shuffle(data)
df = pd.DataFrame(data)[['reviewText', 'overall', 'summary']]
negatives = df[df['overall'] <= 2][:20000]
positives = df[df['overall'] >= 4][:20000]
neutrals = df[df['overall'] == 3][:10000]
df = pd.concat([negatives, neutrals, positives], ignore_index=True)
df = df[df['reviewText'].str.len() <= 15000]
df["embedding"] = df.apply(lambda row: get_embedding(str(row['summary']) + " : " + str(row['reviewText']), engine=embedding_model), axis=1)
df.to_csv("Datasets/CDs_and_Vinyl_Embeddings_50k.csv")
# display(df['overall'].value_counts())

In [63]:
df = pd.read_csv('Datasets/trip_advisor_embeddings_full.csv')
df2 = pd.read_csv('Datasets/CDs_and_Vinyl_Embeddings_50k.csv')
Y_df = pd.concat([df.Rating, df2.overall])
X_df = pd.concat([df.embedding, df2.embedding])
X_df = X_df.apply(literal_eval).apply(np.array)  # convert string to array
# # display(df['Rating'].value_counts())

In [64]:
Y_df = Y_df.apply(lambda x: 0 if x <= 2 else (1 if x == 3 else 2))
X_train, X_test, y_train, y_test = train_test_split(
    list(X_df.values), list(Y_df.values), test_size=0.05, random_state=42
)

In [65]:
# train random forest classifier
clf = RandomForestClassifier(n_estimators=200)
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200)

In [66]:
preds = clf.predict(X_test)
probas = clf.predict_proba(X_test)

report = classification_report(y_test, preds)
print(report)

              precision    recall  f1-score   support

           0       0.75      0.90      0.82      1135
           1       0.76      0.11      0.19       614
           2       0.83      0.97      0.89      1775

    accuracy                           0.80      3524
   macro avg       0.78      0.66      0.63      3524
weighted avg       0.79      0.80      0.75      3524



In [67]:
#inference
clf.predict(np.array(get_embedding("This movie has some of the worst posters in years. This is such amateur composition and photoshopping.", engine=embedding_model)).reshape(1, -1))

array([0], dtype=int64)

In [65]:
kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto").fit(list(df.embedding.values))

In [68]:
Negative = np.array([0, 0, 0])
Neutral = np.array([0, 0, 0])
Positive = np.array([0, 0, 0])
for i in range(len(df)):
    if df.loc[i].Rating == 0:
        Negative[kmeans.labels_[i]] += 1
    if df.loc[i].Rating == 1:
        Neutral[kmeans.labels_[i]] += 1
    if df.loc[i].Rating == 2:
        Positive[kmeans.labels_[i]] += 1

display(Negative)
display(Neutral)
display(Positive)

array([ 20, 400,  45])

array([ 94, 152,  55])

array([1193,  278,  263])